In [1]:
import datetime
import numpy as np
import pandas as pd


In [2]:
### Define input files
positionFile = 'inputs/RSN_Positions_TEAM_20250318.xlsx'
positionSheet = 'RCA History'
positionNameMap = 'params/positionNameMap.csv'
HITLpositionList = 'params/HITLpositionList.csv'
amRepo = 'https://raw.githubusercontent.com/oceanobservatories/asset-management/master'
nodeDeploymentFile = 'https://raw.githubusercontent.com/OOI-CabledArray/deployments/refs/heads/main/NODE_deployments.csv'


In [3]:
### Load positions from RCA xlsx file and create nested dicitonary keyed on position name and time
df_positions = pd.read_excel(positionFile, sheet_name=positionSheet)

### Load HITL positions
df_HITLposition = pd.read_csv(HITLpositionList)
HITLpositionDict = {}
for i in df_HITLposition['referenceDesignator'].unique():
    HITLpositionDict[i] = [{'deployYear':df_HITLposition['deployYear'][j], 
                           'deployNum':df_HITLposition['deployNum'][j], 
                           'positionStartTime':df_HITLposition['positionStartTime'][j],
                            'positionName':df_HITLposition['positionName'][j],
                          } for j in df_HITLposition[df_HITLposition['referenceDesignator']==i].index]

# Initialize the dictionary with empty dictionaries for each name
positionDict = {name: {} for name in set(df_positions['name'])}

# Iterate through each row of the dataframe
for index, row in df_positions.iterrows():
    # Initialize the entry for this name if it doesn't exist
    if row['name'] not in positionDict:
        positionDict[row['name']] = {}

    # Add the data for this position keyed for position start time
    positionDict[row['name']][row['deployed position start'].round('S')] = {
        'year': row['deployed position start'].year, 
        'positionStartTime': row['deployed position start'],
        'positionEndTime': row['deployed position end'],
        'positionTime': row['Position time'],
        'lat': format(row['Latitude'],'.6f'), 
        'lon': format(row['longitude'],'.6f'),
        'waterDepth': row['Seafloor depth (m)'],
        'mooringDepth': row['Mooring top depth (m)'],
        'sourceIndex': index + 2,
    }

### load in position name map dictionary
df_positionMap = pd.read_csv(positionNameMap)
positionMap = dict(zip(df_positionMap['referenceDesignator'],df_positionMap['positionName']))


In [4]:
CabledArray = pd.Series(['CE02SHBP','CE04OSBP','CE04OSPD','CE04OSPS','RS01SBPD','RS01SBPS',
                        'RS01SLBS','RS01SUM1','RS03AXBS','RS03AXPD','RS03AXPS','RS03INT2',
                        'RS03INT1','RS01SUM2','RS03CCAL','RS03ECAL','RS03ASHS','NODES'])

In [5]:
profilers = ['SF0', 'DP0']
# For each deployment, iterate through by row and verify positions
for array in CabledArray:
    print(array)
    if 'NODES' in array:
        deployFile = nodeDeploymentFile
    else:
        deployFile = amRepo + '/deployment/' + array + '_Deploy.csv' 
    df_deploy = pd.read_csv(deployFile, skip_blank_lines = True, comment='#')
    log = []
    for index, row in df_deploy.iterrows():
        positionStartTime = None
        instType = 1
        refDes = row['Reference Designator']
        if any(node in refDes for node in profilers):
            instType = 0
        if 'PC0' in refDes:
            instType = 1
        dt = datetime.datetime.strptime(row['startDateTime'], '%Y-%m-%dT%H:%M:%S')
        deployYear = dt.year
        deployNumber = str(row['deploymentNumber'])

        positionName = positionMap[refDes]
        HITLstart = None
        if refDes in HITLpositionDict.keys():
            HITLstart = [[x['positionStartTime'],x['positionName']] for x in HITLpositionDict[refDes] 
                                 if str(deployYear) in str(x['deployYear']) and deployNumber in str(x['deployNum'])]
            if HITLstart:
                positionStartTime = pd.to_datetime(HITLstart[0][0])
                positionName = HITLstart[0][1]
        if not HITLstart:   
            positions = []
            for k in positionDict[positionName].keys():
                if k.year == deployYear:
                    positions.append(k)
            if positions:
                if len(positions) > 1:
                    print('mulitple positions found...need HITL entry for: ', refDes, deployYear)
                else:
                    positionStartTime = positions[0]
            else:

                positionDates = list(positionDict[positionName].keys())
                positionDateList = list(filter(lambda d: d < dt, positionDates))
                if positionDateList:
                    positionStartTime = min(positionDateList, key = lambda x: abs(x - dt))   
                       
        if positionStartTime:            
            XL_lat = float(positionDict[positionName][positionStartTime]['lat'])
            XL_lon = float(positionDict[positionName][positionStartTime]['lon'])
            XL_waterDepth = abs(int(format(positionDict[positionName][positionStartTime]['waterDepth'],'.0f')))
            if np.isnan(positionDict[positionName][positionStartTime]['mooringDepth']):
                XL_mooringDepth = XL_waterDepth
            else:
                XL_mooringDepth = abs(int(format(positionDict[positionName][positionStartTime]['mooringDepth'],'.0f')))
            XL_sourceIndex = int(positionDict[positionName][positionStartTime]['sourceIndex'])

            logChanges = []
            if row['lat'] != XL_lat:
                df_deploy.at[index, 'lat'] = XL_lat
                logChanges.append('lat')
            if row['lon'] != XL_lon:
                logChanges.append('lon')
                df_deploy.at[index, 'lon'] = XL_lon
            if row['water_depth'] != XL_waterDepth:
                df_deploy.at[index, 'water_depth'] = XL_waterDepth
                logChanges.append('water depth')
            if isinstance(row['deployment_depth'], float):
                if not pd.isna(row['deployment_depth']):
                    df_deploy.at[index, 'deployment_depth'] = int(row['deployment_depth'])
            if isinstance(row['water_depth'], float):
                df_deploy.at[index, 'water_depth'] = int(row['water_depth'])                            
            if instType == 0:
                if 'N/A' not in str(row['deployment_depth']):
                    df_deploy.at[index, 'deployment_depth'] = 'N/A'
                    logChanges.append('deploy depth')
            elif instType == 1:
                if row['deployment_depth'] != XL_mooringDepth:
                    df_deploy.at[index, 'deployment_depth'] = XL_mooringDepth
                    logChanges.append('deploy depth')
            else:
                if row['deployment_depth'] != XL_waterDepth:
                    df_deploy.at[index, 'deployment_depth'] = XL_waterDepth
                    logChanges.append('deploy depth')
            if "The following parameters are preliminary" in str(row['notes']):
                df_deploy.at[index, 'notes'] = ''
            if logChanges:
                logLinePrefix = ','.join([refDes, str(deployYear), str(deployNumber), positionName, str(XL_sourceIndex)])
                logLineChanges = ','.join(logChanges)
                log.append(logLinePrefix + ', ' + logLineChanges)
 
        else:
            print('!!! no positions found in xl or HITL sheets for: ', refDes, deployYear)
            
     
    fileName = 'tmp/AM_deployments/' + array + '_Deploy.csv'
    # Replace NaN values with empty strings and keep 'N/A' as a string
    df_deploy = df_deploy.applymap(lambda x: '' if pd.isna(x) else x)
    df_deploy = df_deploy.applymap(lambda x: int(x) if isinstance(x, float) and x.is_integer() else x)
    df_deploy.to_csv(fileName, index=False, na_rep='', float_format='%.6f')
    
    logFile = 'tmp/AM_deployments/' + array + '_log.csv'
    with open(logFile, 'w') as f:
        f.write('refDes, deployYear, deployNum, positionName, sourceIndex, lineChanges\n')
        for line in log:
            f.write(f"{line}\n")
            

CE02SHBP
CE04OSBP
CE04OSPD
CE04OSPS
RS01SBPD
RS01SBPS
RS01SLBS
RS01SUM1
RS03AXBS
RS03AXPD
RS03AXPS
RS03INT2
RS03INT1
RS01SUM2
RS03CCAL
RS03ECAL
RS03ASHS
NODES
